In [1]:
import time
import datetime
import json
import traceback
import importlib

from functions import *
from twitter import *
from Mayu import *

    
def roop():
    followers = get_followers()
    time.sleep(10)
    timeline = get_timeline(followers)
    if timeline.status_code != 200:
        with open("../error.log", "r") as f:
            l = f.readlines()
            l = str(datetime.datetime.now()) + "\n==========\n\n" + "status code != 200" +"\n==========\n\n\n\n" + "".join(l)
        with open("../error.log", "w") as f:
            print(l, file=f)
            
        
        return True

    
    t = datetime.datetime.now()

    for line in timeline.iter_lines():
        try:
            if (datetime.datetime.now() - t).seconds > 900:
                # breakする
                return True

            stream = line.decode("utf-8")
            raw_data = json.loads(stream)

            mayu = Mayu(raw_data)
            mayu.run()

        except:
            # なんかしらんエラー
            pass

In [ ]:
print("start at", datetime.datetime.now())

while True:
        try:
            roop()
            
        except:
            with open("../error.log", "r") as f:
                l = f.readlines()
                l = str(datetime.datetime.now()) + "\n==========\n\n" + str(traceback.format_exc()) +"\n==========\n\n\n\n" + "".join(l)
            with open("../error.log", "w") as f:
                print(l, file=f)

start at 2018-06-22 10:28:15.166759
